In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

#import data from 2013-2014 NHANES: https://www.kaggle.com/cdc/national-health-and-nutrition-examination-survey
#description of columns: https://wwwn.cdc.gov/nchs/nhanes/2013-2014/demo_h.htm
demo_df = pd.read_csv('../data/demographic.csv')
diet_df = pd.read_csv('../data/diet.csv')
exam_df = pd.read_csv('../data/examination.csv')
labs_df = pd.read_csv('../data/labs.csv')
med_df = pd.read_csv('../data/medications.csv', encoding='latin1')
ques_df = pd.read_csv('../data/questionnaire.csv')

### Step 1: EDA

In [12]:
new_demo_df = demo_df[['SEQN','RIAGENDR','RIDAGEYR','RIDRETH3',
'DMQMILIZ','DMQADFC','DMDBORN4','DMDMARTL']]

In [13]:
new_demo_df

,SEQN,RIAGENDR,RIDAGEYR,RIDRETH3,DMQMILIZ,DMQADFC,DMDBORN4,DMDMARTL
0,73557,1,69,4,1.0,1.0,1,4.0
1,73558,1,54,3,2.0,NaN,1,1.0
2,73559,1,72,3,1.0,1.0,1,1.0
3,73560,1,9,3,NaN,NaN,1,NaN
4,73561,2,73,3,2.0,NaN,1,1.0
...,...,...,...,...,...,...,...,...
10170,83727,1,26,2,2.0,NaN,1,5.0
10171,83728,2,2,1,NaN,NaN,1,NaN
10172,83729,2,42,4,2.0,NaN,2,3.0
10173,83730,1,7,2,NaN,NaN,1,NaN


In [18]:
new_demo_df['RIAGENDR'].apply(lambda x: 'Male' if x == 1 else 'Female')

0        Female
1        Female
2        Female
3        Female
4        Female
          ...  
10170    Female
10171    Female
10172    Female
10173    Female
10174    Female
Name: RIAGENDR, Length: 10175, dtype: object

In [15]:
new_demo_df

,SEQN,RIAGENDR,RIDAGEYR,RIDRETH3,DMQMILIZ,DMQADFC,DMDBORN4,DMDMARTL
0,73557,1,69,4,1.0,1.0,1,4.0
1,73558,1,54,3,2.0,NaN,1,1.0
2,73559,1,72,3,1.0,1.0,1,1.0
3,73560,1,9,3,NaN,NaN,1,NaN
4,73561,2,73,3,2.0,NaN,1,1.0
...,...,...,...,...,...,...,...,...
10170,83727,1,26,2,2.0,NaN,1,5.0
10171,83728,2,2,1,NaN,NaN,1,NaN
10172,83729,2,42,4,2.0,NaN,2,3.0
10173,83730,1,7,2,NaN,NaN,1,NaN
